# A Demo of How To Use ELUC Predictors

In [1]:
import pandas as pd
from sklearn.metrics import mean_absolute_error

from predictors.NeuralNetwork.NeuralNetPredictor import NeuralNetPredictor
from predictors.LinearRegression.LinearRegressionPredictor import LinearRegressionPredictor
from predictors.RandomForest.RandomForestPredictor import RandomForestPredictor
from predictors.evaluator import evaluate_predictor
from data.data import ELUCData
from data import constants

In [2]:
data = ELUCData("projectresilience/ELUC-committed")

### Initialize and fit models a la `sklearn` API

In [3]:
nnp = NeuralNetPredictor(constants.NN_FEATS, epochs=1, train_pct=0.1, hidden_sizes=[128])
nnp.fit(data.train_df.drop("ELUC", axis=1), data.train_df["ELUC"], verbose=True)
print(mean_absolute_error(data.test_df["ELUC"], nnp.predict(data.test_df.drop("ELUC", axis=1))))

  0%|          | 0/1927 [00:00<?, ?it/s]/Users/964840/anaconda3/envs/leaf/lib/python3.10/site-packages/torch/autograd/__init__.py:204: UserWarning: The operator 'aten::sgn.out' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:13.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
100%|██████████| 1927/1927 [00:31<00:00, 61.39it/s]


0.06143991


In [12]:
linreg = LinearRegressionPredictor(features=constants.DIFF_LAND_USE_COLS, n_jobs=-1)
linreg.fit(data.train_df, data.train_df["ELUC"])
print(mean_absolute_error(data.test_df["ELUC"], linreg.predict(data.test_df[constants.DIFF_LAND_USE_COLS])))

0.07567061


In [13]:
rf = RandomForestPredictor(features=constants.NN_FEATS, n_jobs=-1, max_features="sqrt", random_state=42)
rf.fit(data.train_df.loc[2010:], data.train_df.loc[2010:]["ELUC"])
print(mean_absolute_error(data.test_df["ELUC"], rf.predict(data.test_df[constants.NN_FEATS])))

0.045818872993118626


### Save models

In [14]:
nnp.save("predictors/NeuralNetwork/test")
linreg.save("predictors/LinearRegression/test")
rf.save("predictors/RandomForest/test")

### Load model back with exact same path used for save
We can evaluate it as a `Predictor` object in order to evaluate many different types of predictors

In [15]:
nnp_test = NeuralNetPredictor()
nnp_test.load("predictors/NeuralNetwork/test")
linreg_test = LinearRegressionPredictor()
linreg_test.load("predictors/LinearRegression/test")
rf_test = RandomForestPredictor()
rf_test.load("predictors/RandomForest/test")

print(evaluate_predictor(nnp_test, data.test_df, data.test_df["ELUC"]))
print(evaluate_predictor(linreg_test, data.test_df, data.test_df["ELUC"]))
print(evaluate_predictor(rf_test, data.test_df, data.test_df["ELUC"]))

0.060935497
0.07567061
0.045818872993118626


With this flow we can see that we evaluated to the same value!